## Getting videos (don't tell em!)

In [ ]:
!pip install youtube-dl
!add-apt-repository ppa:savoury1/ffmpeg4 -y
!add-apt-repository ppa:savoury1/graphics -y
!add-apt-repository ppa:savoury1/multimedia -y
!apt-get update
!apt-get upgrade && sudo apt-get dist-upgrade
!apt-get install ffmpeg
!rm -rf orig.mp4
!rm -rf in.mp4
!youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best -o orig.mp4 https://www.youtube.com/watch?v=LrzFldhHGhc
!ffmpeg -i orig.mp4 -filter:v "crop=720:1080:600:0" -c:a copy in.mp4

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

## Install dependencies

In [ ]:
!pip install ffmpeg-python
import ffmpeg
import numpy as np
from ipywidgets import interact
from matplotlib import pyplot as plt
import ipywidgets as widgets
import math
import time

## Do the stuff (the hard part)

- Get each frame
- Get pixels needed to build the original source (256x384)
- Upscale using Nearest Neighbor to nearest round number multiplier (x5, 1280x1920)
- Upscale using Bilinear to the target resolution (1440x2160)

In [ ]:
print("Preparing variables...")

probe = ffmpeg.probe('in.mp4')
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

input = "in.mp4"
output = "out.mp4"
output_real = "real.mp4"
width_input = int(video_info['width'])
height_input = int(video_info['height'])
width_real = 256
height_real = 384
# width_output = 960
# height_output = 1440
width_output = 2160
height_output = 3240
start_time = time.time()
frame_count = int(video_info['nb_frames'])
frame_curr = 0
section_start_time = time.time()

width_real_target = []
height_real_target = []
width_real_increment = width_input/width_real
height_real_increment = height_input/height_real
width_real_middle = width_real_increment/2
height_real_middle = height_real_increment/2

for i in range(width_real):
  width_real_target.append(width_real_middle + width_real_increment * i)

for i in range(height_real):
  height_real_target.append(height_real_middle + height_real_increment * i)

width_real_target = list(map(lambda a: math.floor(a), width_real_target))
height_real_target = list(map(lambda a: math.floor(a), height_real_target))

multiplier = math.floor(width_output/width_real)

print(f"Input resolution is {width_input}x{height_input}")
print(f"Real resolution is {width_real}x{height_real}")
print(f"Multiplier is {multiplier} ({width_real*multiplier}x{height_real*multiplier})")
print(f"Output resolution is {width_output}x{height_output}")

process1 = (
  ffmpeg
  .input(input)
  .output(
    'pipe:', 
    format='rawvideo', 
    pix_fmt='rgb24', 
    # vframes=600
    )
  .run_async(pipe_stdout=True)
)

process2 = (
  ffmpeg
  .input(
    'pipe:', 
    format='rawvideo', 
    pix_fmt='rgb24', 
    s='{}x{}'.format(width_real, height_real), 
    framerate=60000/1001,
  )
  .output(
    'pipe:', 
    format='rawvideo', 
    pix_fmt='rgb24', 
    s='{}x{}'.format(width_real*multiplier, height_real*multiplier), 
    sws_flags="neighbor"
  )
  .run_async(pipe_stdout=True, pipe_stdin=True)
)

process3 = (
  ffmpeg
  .input(
    'pipe:', 
    format='rawvideo', 
    pix_fmt='rgb24', 
    s='{}x{}'.format(width_real*multiplier, height_real*multiplier), 
    framerate=60000/1001
  )
  .output(
    output, 
    pix_fmt='yuv420p', 
    s='{}x{}'.format(width_output, height_output), 
    sws_flags="bilinear", 
    vcodec="h264_nvenc", 
    crf=18, 
    preset='default', 
    tune="fastdecode",
    # vcodec="libx264", 
    # crf=18, 
    # preset='fast', 
  )
  .overwrite_output()
  .run_async(pipe_stdin=True)
)

process4 = (
  ffmpeg
  .input(
    'pipe:', 
    format='rawvideo', 
    s='{}x{}'.format(width_real, height_real), 
    pix_fmt='rgb24', 
    framerate=60000/1001,
  )
  .output(
    output_real, 
    pix_fmt='yuv420p', 
    s='{}x{}'.format(width_real, height_real), 
    vcodec="h264_nvenc", 
    crf=24, 
    preset='default', 
    tune="fastdecode",
    # vcodec="libx264", 
    # crf=14, 
    # preset='ultrafast', 
  )
  .overwrite_output()
  .run_async(pipe_stdin=True)
)

print("Encoding...")

while True:
  in_bytes = process1.stdout.read(width_input * height_input * 3)
  if not in_bytes:
    break
  in_frame = (
    np
    .frombuffer(in_bytes, np.uint8)
    .reshape([height_input, width_input, 3])
  )

  real_frame = np.zeros((height_real, width_real, 3), dtype=np.uint8)
  # print(real_frame)

  x_real = -1
  y_real = -1
  for y in height_real_target:
    y_real += 1
    x_real = -1
    for x in width_real_target:
      x_real += 1
      real_frame[y_real][x_real] = in_frame[y][x]

  # print(real_frame)

  # out_frame = (
      
  # )

  # See examples/tensorflow_stream.py:
  # out_frame = deep_dream.process_frame(in_frame)

  process2.stdin.write(
    real_frame
    .astype(np.uint8)
    .tobytes()
  )
  
  process4.stdin.write(
    real_frame
    .astype(np.uint8)
    .tobytes()
  )

  process3.stdin.write(
    process2
    .stdout
    .read(width_real*multiplier * height_real*multiplier * 3)
  )

  frame_curr += 1
  if math.floor(frame_curr/100)*100 == frame_curr:
    section_time = time.time() - section_start_time
    run_time = time.time() - start_time
    frame_remaning = frame_count - frame_curr
    est = (section_time/100)*frame_remaning
    print(f"Frame {frame_curr} of {frame_count}. ({round((frame_curr/frame_count)*100, 2)}%, sect.: {math.floor(section_time/60)}m{math.floor(section_time%60)}s, all: {math.floor(run_time/60)}m{math.floor(run_time%60)}s, est.: {math.floor(est/60)}m{math.floor(est%60)}s)")
    section_start_time = time.time()


print(f"Frame {frame_curr} of {frame_count}.")

process2.stdin.close()
process3.stdin.close()
process4.stdin.close()
process1.wait()
process2.wait()
process3.wait()
process4.wait()

run_time = time.time() - start_time

print(f"All done!")
print(f"Done in {math.floor(run_time/60)}m{math.floor(run_time%60)}s.")

Preparing variables...
Input resolution is 720x1080
Real resolution is 256x384
Multiplier is 8 (2048x3072)
Output resolution is 2160x3240
Encoding...
Frame 100 of 15910. (0.63%, sect.: 0m17s, all: 0m17s, est.: 47m6s)
Frame 200 of 15910. (1.26%, sect.: 0m17s, all: 0m35s, est.: 44m54s)
Frame 300 of 15910. (1.89%, sect.: 0m17s, all: 0m52s, est.: 44m51s)
Frame 400 of 15910. (2.51%, sect.: 0m17s, all: 1m9s, est.: 44m22s)
Frame 500 of 15910. (3.14%, sect.: 0m17s, all: 1m26s, est.: 44m20s)
Frame 600 of 15910. (3.77%, sect.: 0m17s, all: 1m43s, est.: 44m0s)
Frame 700 of 15910. (4.4%, sect.: 0m17s, all: 2m1s, est.: 43m55s)
Frame 800 of 15910. (5.03%, sect.: 0m17s, all: 2m18s, est.: 43m28s)
Frame 900 of 15910. (5.66%, sect.: 0m17s, all: 2m35s, est.: 43m14s)
Frame 1000 of 15910. (6.29%, sect.: 0m17s, all: 2m53s, est.: 42m45s)
Frame 1100 of 15910. (6.91%, sect.: 0m17s, all: 3m10s, est.: 42m41s)
Frame 1200 of 15910. (7.54%, sect.: 0m17s, all: 3m27s, est.: 42m41s)
Frame 1300 of 15910. (8.17%, sect.: 

In [ ]:
# plt.imshow(real_frame[:,:,:])

## Add audio

In [ ]:
!ffmpeg -i out.mp4 -i in.mp4 -c copy -map 0:v:0 -map 1:a:0 -shortest out_audio.mkv
!ffmpeg -i real.mp4 -i in.mp4 -c copy -map 0:v:0 -map 1:a:0 -shortest real_audio.mkv

ffmpeg version 4.3.2-2ubuntu0~18.04.sav0 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='2ubuntu0~18.04.sav0' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --en

## Upscale naively using nearest neighbor

In [15]:
!ffmpeg -i orig.mp4 -c:a copy -vf "crop=720:1080:600:0, scale=2160x3240" out_naive.mp4




ffmpeg version 4.3.2-2ubuntu0~18.04.sav0 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='2ubuntu0~18.04.sav0' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --en

## Generate real source only

In [ ]:
# print("Preparing variables...")

# probe = ffmpeg.probe('in.mp4')
# video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

# input = "in.mp4"
# output = "out_real.mp4"
# width_input = int(video_info['width'])
# height_input = int(video_info['height'])
# width_real = 256
# height_real = 384
# start_time = time.time()
# frame_count = int(video_info['nb_frames'])
# frame_curr = 0

# width_real_target = []
# height_real_target = []
# width_real_increment = width_input/width_real
# height_real_increment = height_input/height_real
# width_real_middle = width_real_increment/2
# height_real_middle = height_real_increment/2

# for i in range(width_real):
#   width_real_target.append(width_real_middle + width_real_increment * i)

# for i in range(height_real):
#   height_real_target.append(height_real_middle + height_real_increment * i)

# width_real_target = list(map(lambda a: math.floor(a), width_real_target))
# height_real_target = list(map(lambda a: math.floor(a), height_real_target))

# process1 = (
#     ffmpeg
#     .input(input)
#     .output(
#       'pipe:', 
#       format='rawvideo', 
#       pix_fmt='rgb24', 
#       vframes=600
#       )
#     .run_async(pipe_stdout=True)
# )

# process2 = (
#     ffmpeg
#     .input(
#       'pipe:', 
#       format='rawvideo', 
#       s='{}x{}'.format(width_real, height_real), 
#       pix_fmt='rgb24', 
#       framerate=60000/1001,
#     )
#     .output(
#       output, 
#       pix_fmt='yuv420p', 
#       s='{}x{}'.format(width_real, height_real), 
#       # vcodec="h264_nvenc", 
#       # crf=18, 
#       # preset='default', 
#       vcodec="libx264", 
#       crf=14, 
#       preset='ultrafast', 
#     )
#     .overwrite_output()
#     .run_async(pipe_stdin=True)
# )

# print("Encoding...")

# while True:
#   in_bytes = process1.stdout.read(width_input * height_input * 3)
#   if not in_bytes:
#     break
#   in_frame = (
#     np
#     .frombuffer(in_bytes, np.uint8)
#     .reshape([height_input, width_input, 3])
#   )

#   real_frame = np.zeros((height_real, width_real, 3), dtype=np.uint8)
#   # print(real_frame)

#   y_real = -1
#   for y in height_real_target:
#     y_real += 1
#     x_real = -1
#     for x in width_real_target:
#       x_real += 1
#       real_frame[y_real][x_real] = in_frame[y][x]

#   # print(real_frame)

#   # out_frame = (
      
#   # )

#   # See examples/tensorflow_stream.py:
#   # out_frame = deep_dream.process_frame(in_frame)

#   process2.stdin.write(
#       real_frame
#       .astype(np.uint8)
#       .tobytes()
#   )
  
#   # process3.stdin.write(process2.stdout.read(width_real*multiplier * height_real*multiplier * 3))

#   frame_curr += 1
#   if math.floor(frame_curr/100)*100 == frame_curr:
#     print(f"Frame {frame_curr} of {frame_count}")

# print(f"Frame {frame_curr} of {frame_count}")

# process2.stdin.close()
# process1.wait()
# process2.wait()

# print(f"All done!")
# print(f"Done in {time.time() - start_time} seconds.")